In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.8.0'

In [5]:
co = {}
from qkeras.utils import _add_supported_quantized_objects
_add_supported_quantized_objects(co)
model = tf.keras.models.load_model("../model/qmodel.h5", custom_objects=co)

In [6]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (QDense)                (None, 4)                 8         
                                                                 
 relu1 (QActivation)         (None, 4)                 0         
                                                                 
 fc2e (QDense)               (None, 8)                 40        
                                                                 
 relu2e (QActivation)        (None, 8)                 0         
                                                                 
 fc3 (QDense)                (None, 11)                99        
                                                                 
 relu3 (QActivation)         (None, 11)                0         
                                                                 
 fc3e (QDense)               (None, 11)              

In [55]:
from qkeras.quantizers import get_quantizer

In [58]:
# Manually requantize a layer
def requantize(layer, bits, integer_bits):
    quantizers = [layer.kernel_quantizer, layer.bias_quantizer]
    new_quantizers = []
    for quantizer in quantizers:
        quantizer["config"]["bits"] = bits
        quantizer["config"]["integer"] = integer_bits
        new_quantizers.append(get_quantizer(quantizer))
    layer.quantizers = new_quantizers


In [65]:
layer_fc3 = model.layers[4]
layer_output = model.layers[-1]
requantize(layer_fc3, 10, 0)
requantize(layer_fc3, 12, 0)

In [9]:
import pandas as pd
source_data = pd.read_csv("../data/xrotate.csv", header=None)
source_data.drop(0, inplace=True) # Drop -pi row for evaluation too

In [10]:
X,Y = source_data.iloc[:,20], source_data.iloc[:,:20]

In [15]:
import hls4ml

In [16]:
hls4ml.__version__

'0.6.0'

In [17]:
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

In [18]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: fc2e, layer type: QDense
Layer name: relu2e, layer type: QActivation
Layer name: fc3, layer type: QDense
Layer name: relu3, layer type: QActivation
Layer name: fc3e, layer type: QDense
Layer name: relu3e, layer type: QActivation
Layer name: fc3e2, layer type: QDense
Layer name: relu3e2, layer type: QActivation
Layer name: fc3e3, layer type: QDense
Layer name: relu3e3, layer type: QActivation
Layer name: output, layer type: QDense


In [19]:
hls_model = hls4ml.converters.convert_from_keras_model(model, hls_config=config, output_dir='../model/hlsout', part='XC7A100T')

Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: InputLayer, input shapes: [[None, 1]], output shape: [None, 1]
Layer name: fc1, layer type: QDense, input shapes: [[None, 1]], output shape: [None, 4]
Layer name: relu1, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc2e, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 8]
Layer name: relu2e, layer type: Activation, input shapes: [[None, 8]], output shape: [None, 8]
Layer name: fc3, layer type: QDense, input shapes: [[None, 8]], output shape: [None, 11]
Layer name: relu3, layer type: Activation, input shapes: [[None, 11]], output shape: [None, 11]
Layer name: fc3e, layer type: QDense, input shapes: [[None, 11]], output shape: [None, 11]
Layer name: relu3e, layer type: Activation, input shapes: [[None, 11]], output shape: [None, 11]
Layer name: fc3e2, layer type: QDense, input shapes: [[None, 11]], output shape: [None, 11]
Layer name: relu3e2, layer type: Acti

In [ ]:
hls_model.compile()

# Now, let's obtain alpha values from this model

In [20]:
import numpy as np

In [21]:
betas = list(X)

In [25]:
def mse(a, b):
     assert(len(a)==len(b))
     accum = 0
     for i in range(len(a)):
             accum += (a[i]-b[i])**2
     return accum/len(a)

def eval_model_hls(m):
    inputs = betas
    outputs = Y.iloc
    accum = 0
    for i in range(len(inputs)):
        real = outputs[i]
        pred = m.predict(np.ascontiguousarray(inputs[i])).tolist()
        accum += mse(real, pred)
    return accum / len(inputs)

In [ ]:
eval_model_hls(hls_model)

In [ ]:
# For each beta, output a file with:
# - Predicted alpha
# - Beta value
# - Training set alpha
for i in range(len(X)):
    predicted = hls_model.predict(np.ascontiguousarray(betas[i])).tolist()
    beta = betas[i]
    real = list(Y.iloc[i])
    out_arr = predicted + [beta] + real + [""]
    out_arr = [str(x) for x in out_arr]
    print("\n".join(out_arr))
    assert(len(out_arr) == 20+1+20+1)
    with open("../scratch/hls-outputs/tfout_" + str(i) + ".txt", "w") as f:
        f.write("\n".join(out_arr))